In [1]:
import pandas as pd
import numpy as np

In [2]:
! pip install -e ..

Obtaining file:///Users/s2341012/Dropbox/DISMaL_chapter/DISMaL
  Preparing metadata (setup.py) ... done
  Attempting uninstall: dismal
    Found existing installation: dismal 0.1
    Uninstalling dismal-0.1:
      Successfully uninstalled dismal-0.1
  Running setup.py develop for dismal


In [3]:
from dismal import segregating_sites, preprocess

In [4]:
callset = preprocess.CallSet("../test_data/brenthis_500K.vcf.gz", "../test_data/vcf.npz")
blocks = preprocess.gff3_to_blocks("../test_data/brenthis_head1000.gff3", blocklen=100, min_block_distance=10_000, parquet_path="../test_data/blocks.parquet.gz")
sample_map = pd.read_csv("../test_data/samples_map.csv")

In [5]:
numeric_chromosomes = np.unique(callset.chromosomes, return_inverse=True)[1]
block_start_idxs = np.array(blocks["start"])

In [6]:
callset_indexing = pd.DataFrame({"sample": callset.samples, "idx": [i for i in range(0, len(callset.samples))]})
pops = sample_map.merge(callset_indexing, on="sample")["population"]

pop1_idx = np.where(pops == "BI")[0]
pop2_idx = np.where(pops == "BD")[0]

In [7]:
%%timeit

segregating_sites.blockwise_segregating_sites(block_start_idxs=block_start_idxs,
                                              blocklen=100,
                                              gt=np.array(callset.gt),
                                              chroms=numeric_chromosomes,
                                              snp_pos=callset.pos,
                                              pop1_idxs=pop1_idx,
                                              pop2_idxs=pop2_idx)

4.25 ms ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
preprocess.blockwise_segregating_sites(blocks, callset, samples=sample_map)

100%|██████████| 35/35 [00:00<00:00, 2027.47it/s]

19.6 ms ± 416 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
block_start_idxs.dtype

dtype('int64')